<a href="https://colab.research.google.com/github/luanps/pyserini/blob/master/Run_pyserini_distillbert_kd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Distilbert-KD Passage Ranking on MSMARCO

Sebastian Hofstätter, Sophia Althammer, Michael Schröder, Mete Sertkan, and Allan Hanbury. Improving Efficient Neural Ranking Models with Cross-Architecture Knowledge Distillation. arXiv:2010.02666, October 2020.



## Install dependencies

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
%%capture
!pip install pyserini

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [3]:
!pip install faiss-cpu

     |████████████████████████████████| 8.6 MB 9.3 MB/s 


## Dense retrieval with Distilbert-KD - MSMARCO Passage Ranking

In [ ]:
!python -m pyserini.dsearch --topics msmarco-passage-dev-subset \
                            --index msmarco-passage-distilbert-dot-margin_mse-T2-bf \
                            --encoded-queries distilbert_kd-msmarco-passage-dev-subset \
                            --batch-size 36 \
                            --threads 12 \
                            --output run.msmarco-passage.distilbert-dot-margin_mse-T2.bf.tsv \
                            --output-format msmarco

Using pre-defined topic order for msmarco-passage-dev-subset
Attempting to initialize pre-encoded queries distilbert_kd-msmarco-passage-dev-subset.
query-embedding-distilbert_kd-msmarco-passage-dev-subset-20210419-9323ec.tar.gz: 19.1MB [00:01, 16.0MB/s]                
Extracting /root/.cache/pyserini/queries/query-embedding-distilbert_kd-msmarco-passage-dev-subset-20210419-9323ec.tar.gz into /root/.cache/pyserini/queries/query-embedding-distilbert_kd-msmarco-passage-dev-subset-20210419-9323ec.4706ec91183eefa9771e9311fe4799e0...
Initializing distilbert_kd-msmarco-passage-dev-subset...
Attempting to initialize pre-built index msmarco-passage-distilbert-dot-margin_mse-T2-bf.
dindex-msmarco-passage-distilbert-dot-margin_mse-T2-20210316-d44c3a.tar.gz:  52% 12.2G/23.4G [08:20<07:50, 25.6MB/s]

In [ ]:
!gsutil cp run.msmarco-passage.distilbert-dot-margin_mse-T2.bf.tsv gs://luanps/information_retrieval/pyserini/distilbert-kd/

### Evaluation

In [ ]:
#MRR Eval
!python -m pyserini.eval.msmarco_passage_eval msmarco-passage-dev-subset run.msmarco-passage.distilbert-dot-margin_mse-T2.bf.tsv >> msmarco_passage_distilbert-dot-margin_mse-T2.bf_mrr_eval.txt

In [ ]:
#TREC Eval
!python -m pyserini.eval.convert_msmarco_run_to_trec_run --input run.msmarco-passage.distilbert-dot-margin_mse-T2.bf.tsv \
                                                         --output run.msmarco-passage.distilbert-dot-margin_mse-T2.bf.trec

!python -m pyserini.eval.trec_eval -c -mrecall.1000 \
                                      -mmap msmarco-passage-dev-subset \
                                      run.msmarco-passage.distilbert-dot-margin_mse-T2.bf.trec >> msmarco_passage_distilbert-dot-margin_mse-T2_eval.txt

In [ ]:
!gsutil cp  msmarco_passage_distilbert-dot-margin_mse-T2.bf_mrr_eval.txt msmarco_passage_distilbert-dot-margin_mse-T2_eval.txt gs://luanps/information_retrieval/pyserini/distilbert-kd/